In [78]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings


warnings.filterwarnings("ignore")

df311 = pd.read_csv(r"C:\Users\rebel\UE_work\Urban_Echo\311_cleaned.csv")

df311.head()

,City Council District,Department,Service Request Type,ERT (Estimated Response Time),Status,Priority,Zip Code
0,2,Code Compliance,CCS,4,Closed,Standard,75226.0
1,5,Water Utilities,DWU,5,Closed (Duplicate),Standard,75217.0
2,4,Code Compliance,CCS,4,Closed,Standard,75216.0
3,3,Code Compliance,CCS,4,Closed,Standard,75241.0
4,8,Code Compliance,CCS,4,Closed,Standard,75217.0


In [79]:
print(df311.columns.tolist())

['City Council District', 'Department', 'Service Request Type', 'ERT (Estimated Response Time)', 'Status', 'Priority', 'Zip Code']


In [46]:
#df311 = df311[~df311['City Council District'].astype(str).str.contains(',')]


In [80]:
#df311 = df311[df311["City Council District"].str.contains(",") == False]
df311['City Council District'] = df311['City Council District'].astype('category')
#df311['Address'] = df311['Address'].astype('category')
df311['Department'] = df311['Department'].astype('category')
df311['Service Request Type'] = df311['Service Request Type'].astype('category')
df311['ERT (Estimated Response Time)'] = df311['ERT (Estimated Response Time)'].astype('int64')
df311['Status'] = df311['Status'].astype('category')
df311['Priority'] = df311['Priority'].astype('category')
df311['Zip Code'] = df311['Zip Code'].astype('int64')
# df311['Unique Key'] = df311['Priority'].astype('category')
# df311['Lat_Long Location'] = df311['Lat_Long Location'].astype('category')


# removed and delegated to cleaning

# df311 = df311.drop(columns='Lat_Long Location')
# df311 = df311.drop(columns= 'Unique Key')
# # df311 = df311.drop(columns="Address")

print(df311.dtypes)

City Council District            category
Department                       category
Service Request Type             category
ERT (Estimated Response Time)       int64
Status                           category
Priority                         category
Zip Code                            int64
dtype: object


In [81]:
print(df311.columns.tolist())

['City Council District', 'Department', 'Service Request Type', 'ERT (Estimated Response Time)', 'Status', 'Priority', 'Zip Code']


In [82]:
df311.describe(exclude=np.number)

,City Council District,Department,Service Request Type,Status,Priority
count,1165511,1165511,1165511,1165511,1165511
unique,14,29,47,14,7
top,4,Code Compliance,CCS,Closed,Standard
freq,123603,400138,399559,1075343,954853


In [50]:
# convert the dtypes from object type to float, int64, etc.
# There are string values contained within this column - change the 
# 311_cleaned.csv file
#df311["City Council District"].astype(np.int64)

In [83]:
from sklearn.model_selection import train_test_split

# Extract feature and target arrays
X, y = df311.drop('ERT (Estimated Response Time)', axis=1), df311[['ERT (Estimated Response Time)']]

In [84]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
#print(X_train.dtypes)

In [85]:
# Assuming df311['City Council District'] contains your categorical labels
# Instantiate LabelEncoder

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Fit label encoder and transform the labels
df311['City Council Districtt'] = label_encoder.fit_transform(df311['City Council District'])
df311['Department'] = label_encoder.fit_transform(df311['Department'])
df311['Service Request Type'] = label_encoder.fit_transform(df311['Service Request Type'])
df311['Status'] = label_encoder.fit_transform(df311['Status'])
df311['Priority'] = label_encoder.fit_transform(df311['Priority'])

# dtrain_clf = xgb.DMatrix(data=X_train, label=encoded_labels[:len(X_train)], enable_categorical=True)
# dtest_clf = xgb.DMatrix(data=X_test, label=encoded_labels[len(X_train):], enable_categorical=True)


In [86]:
import xgboost as xgb

# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [87]:
import numpy as np 

In [93]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 10000

evals = [(dtest_reg, "validation"), (dtrain_reg, "train")]


model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
   early_stopping_rounds=500,
   verbose_eval=250
)

[0]	validation-rmse:1.75802	train-rmse:1.75094
[250]	validation-rmse:1.48198	train-rmse:1.46409
[500]	validation-rmse:1.48303	train-rmse:1.46243
[750]	validation-rmse:1.48365	train-rmse:1.46196
[1000]	validation-rmse:1.48416	train-rmse:1.46172
[1250]	validation-rmse:1.48452	train-rmse:1.46158
[1500]	validation-rmse:1.48469	train-rmse:1.46152
[1750]	validation-rmse:1.48483	train-rmse:1.46147
[2000]	validation-rmse:1.48494	train-rmse:1.46145
[2250]	validation-rmse:1.48506	train-rmse:1.46143
[2500]	validation-rmse:1.48512	train-rmse:1.46142
[2750]	validation-rmse:1.48519	train-rmse:1.46141
[3000]	validation-rmse:1.48524	train-rmse:1.46140
[3250]	validation-rmse:1.48527	train-rmse:1.46139
[3500]	validation-rmse:1.48528	train-rmse:1.46139
[3750]	validation-rmse:1.48528	train-rmse:1.46139
[3895]	validation-rmse:1.48528	train-rmse:1.46139


In [94]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 1000

results = xgb.cv(
   params, dtrain_reg,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)

In [95]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,1.750931,0.001056,1.751055,0.004687
1,1.624217,0.000983,1.624553,0.004414
2,1.557276,0.001016,1.557933,0.003997
3,1.522364,0.000879,1.523184,0.003936
4,1.504356,0.000831,1.505330,0.003812


In [96]:
results.keys()

Index(['train-rmse-mean', 'train-rmse-std', 'test-rmse-mean', 'test-rmse-std'], dtype='object')

In [107]:
#we want the test-rmse-mean here 
#because it is important to assess how well the model is expected to behave in a real world 
# scenario. 
results['test-rmse-mean'].mean(axis=0)
column_range = df311['ERT (Estimated Response Time)'].max() - df311['ERT (Estimated Response Time)'].min()

print("column range ", column_range)

print(results['test-rmse-mean'].mean(axis=0))

print(df311['ERT (Estimated Response Time)'].median())
print(df311['ERT (Estimated Response Time)'].mean())

column range  13
1.4829684994404417
4.0
3.8687674333403974


In [111]:
import matplotlib.pyplot as plt

# Train your XGBoost model
# Assuming you have your dmatrix objects for training data and labels
# Let's call them dtrain
xg_reg = xgb.train(params, dtrain_reg, num_boost_round=10000)

importance_type = 'weight'  # Can be 'weight', 'gain', or 'cover'
feature_importances = xg_reg.get_score(importance_type=importance_type)

In [112]:
print(feature_importances)

{'City Council District': 34828.0, 'Department': 24458.0, 'Service Request Type': 14107.0, 'Status': 26332.0, 'Priority': 16599.0, 'Zip Code': 63168.0}


In [115]:
total_importances = sum(feature_importances.values())

# for feature, importance in 
# print(feature_importances / total_importances * 100)
print(total_importances)

for feature, importance in feature_importances.items(): 
    importance_percentage = (importance / total_importances) * 100
    print(f"{feature}: {importance_percentage:.2f}%") 

179492.0
City Council District: 19.40%
Department: 13.63%
Service Request Type: 7.86%
Status: 14.67%
Priority: 9.25%
Zip Code: 35.19%


In [56]:
# # Define hyperparameters
# params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
# n = 100
# model = xgb.train(
#    params=params,
#    dtrain=dtrain_reg,
#    num_boost_round=n,
# )

In [57]:
# #evaluation
# # from sklearn.metrics import mean_squared_errors

# preds = model.predict(dtest_reg)

In [58]:
# rmse = mean_squared_error(y_test, preds, squared=False)

# print(f"RMSE of the base model: {rmse:.3f}")

In [59]:
# evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]
# model = xgb.train(
#    params=params,
#    dtrain=dtrain_reg,
#    num_boost_round=n,
#    evals=evals,
#    verbose_eval=500,
#    early_stopping_rounds=50
# )

In [60]:
# params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
# n = 500

# results = xgb.cv(
#    params, dtrain_reg,
#    num_boost_round=n,
#    nfold=5,
#    early_stopping_rounds=20
# )

# # This gives us the results of the spilts
# results.head()

# # We want to find the minimum rmse-mean 
# best_rmse = results['test-rmse-mean'].min()

# best_rmse



In [61]:
# len(np.unique(df311['City Council District']))

In [62]:
columns = df311.columns
for col in columns: 
    print(f"col = {col} {len(np.unique(df311[col]))}")

col = City Council District 14
col = Department 29
col = Service Request Type 47
col = ERT (Estimated Response Time) 9
col = Status 14
col = Priority 7
col = Zip Code 74


In [63]:
# params = {"objective": "multi:softprob", "tree_method": "gpu_hist", "num_class": len(np.unique(df311['City Council District']))}
# n = 1000

# results = xgb.cv(
#    params, dtrain_clf,
#    num_boost_round=n,
#    nfold=5,
#    early_stopping_rounds=100,
#    metrics=["mlogloss", "auc", "merror"],
# )
